# 19. Multi-Layer Perceptron (MLP) - Part 3
I want to try it out on the main categories as well to see what happens to the results. This gave us the best result with LinearSVC.

## Preprocessing

In [1]:
import pandas as pd
from preprocessing import PreProcessor

pp = PreProcessor()

df = pd.read_csv('Structured_DataFrame_Main_Categories.csv', index_col=0)
df['Item Description'] = df['Item Description'].apply(lambda d: pp.preprocess(str(d)))
df

,Category,Item Description,category_id
0,Services,month huluplu gift code month huluplu code wor...,0
1,Services,pay tv sky uk sky germani hd tv much cccam ser...,0
2,Services,offici account creator extrem tag submiss fix ...,0
3,Services,vpn tor sock tutori setup vpn tor sock super s...,0
4,Services,facebook hack guid guid teach hack facebook ac...,0
...,...,...,...
109585,Drugs,gr purifi opium list gramm redefin opium pefec...,1
109586,Weapons,ship ticket order ship one gun bought must bou...,11
109587,Drugs,gram white afghani heroin full escrow gram whi...,1
109588,Drugs,gram white afghani heroin full escrow gram whi...,1


## Vectorizing

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2))
features = tfidf.fit_transform(df['Item Description'])
labels = df.Category

features

<109563x102238 sparse matrix of type '<class 'numpy.float64'>'
	with 3414110 stored elements in Compressed Sparse Row format>

## Splitting

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=0)

X_train

<73407x102238 sparse matrix of type '<class 'numpy.float64'>'
	with 2286840 stored elements in Compressed Sparse Row format>

## Creating the model
Compared to the part 1 MLP, I decreased the tolerance and max_iter so the model would stop earlier, because this one trains a lot slower due to the larger amount of data.

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=250, learning_rate='constant', 
                     alpha=0.001, solver='adam', verbose=1, random_state=21, tol=0.000001)
clf

MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=250, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=21, shuffle=True, solver='adam', tol=1e-06,
       validation_fraction=0.1, verbose=1, warm_start=False)

## Training the model

In [5]:
clf.fit(X_train, y_train)

Iteration 1, loss = 0.49147250
Iteration 2, loss = 0.14836864
Iteration 3, loss = 0.08407493
Iteration 4, loss = 0.06094960
Iteration 5, loss = 0.04935979
Iteration 6, loss = 0.04430022
Iteration 7, loss = 0.04103929
Iteration 8, loss = 0.03844076
Iteration 9, loss = 0.03646627
Iteration 10, loss = 0.03457898
Iteration 11, loss = 0.03345786
Iteration 12, loss = 0.03327794
Iteration 13, loss = 0.03373274
Iteration 14, loss = 0.03259046
Iteration 15, loss = 0.03209296
Iteration 16, loss = 0.03158469
Iteration 17, loss = 0.03043143
Iteration 18, loss = 0.02991386
Iteration 19, loss = 0.03002192
Iteration 20, loss = 0.02853127
Iteration 21, loss = 0.02929078
Iteration 22, loss = 0.02884922
Iteration 23, loss = 0.02806576
Iteration 24, loss = 0.02810721
Iteration 25, loss = 0.02860418
Iteration 26, loss = 0.02691465
Iteration 27, loss = 0.02665974
Iteration 28, loss = 0.02653108
Iteration 29, loss = 0.02631287
Iteration 30, loss = 0.02678800
Iteration 31, loss = 0.02628383
Iteration 32, los

/opt/jupyterhub/anaconda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=250, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=21, shuffle=True, solver='adam', tol=1e-06,
       validation_fraction=0.1, verbose=1, warm_start=False)

## Validating the model

In [6]:
y_pred = clf.predict(X_test)

In [7]:
accuracy_score(y_test, y_pred)

0.9480307556145592

## Save the model

In [8]:
import pickle

filename = 'MLP_Main_Model.sav'
pickle.dump(clf, open(filename, 'wb'))

# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9480307556145592


## Conclusion
The model performs quite well on the dataset. In notebook 21 I will compare some scores of the entire dataset and main categories to this one and other methods.